In [31]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler

In [32]:
# Load the dataset
data = pd.read_csv("NSSO68.csv")

# Selecting relevant columns for analysis
selected_cols = ["MPCE_URP", "Age", "Sex", "Education", "Religion", "hhdsz"]
data_selected = data[selected_cols]

# Handling missing values if any
data_selected = data_selected.dropna()

# Define censored data (left and right bounds)
censored = (data_selected["MPCE_URP"] == 0)  # Assuming censoring at 0

# Scale the data
scaler = StandardScaler()
data_selected_scaled = scaler.fit_transform(data_selected.drop(columns=["MPCE_URP"]))

C:\Users\user\AppData\Local\Temp\ipykernel_15764\786332066.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("NSSO68.csv")


In [33]:
# Define the Tobit likelihood function
def tobit_likelihood(params, y, X, censored):
    beta = params[:-2]
    sigma = params[-2]
    nu = params[-1]
    
    mu = np.dot(X, beta)
    resid = (y - mu) / sigma
    pdf = norm.pdf(resid)
    cdf = norm.cdf(nu)
    
    LL = np.sum(np.log(np.where(censored, cdf, pdf / sigma)))
    return -LL


In [34]:
# Initial guess for parameters
mean_y = np.mean(data_selected["MPCE_URP"])
initial_params = np.hstack((np.zeros(data_selected.shape[1]), [1.0, mean_y]))

# Exogenous variables
X = sm.add_constant(data_selected_scaled)

# Endogenous variable
y = data_selected["MPCE_URP"]

In [35]:

# Estimate parameters using maximum likelihood estimation (MLE)
results = opt.minimize(tobit_likelihood, initial_params, args=(y, X, censored), method='BFGS')

# Extract estimated parameters
beta_est = results.x[:-2]
sigma_est = results.x[-2]
nu_est = results.x[-1]


C:\Users\user\AppData\Local\Temp\ipykernel_15764\4082158152.py:12: RuntimeWarning: divide by zero encountered in log
  LL = np.sum(np.log(np.where(censored, cdf, pdf / sigma)))
C:\Users\user\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [36]:
# Print results
print("Coefficients:")
print(pd.DataFrame({"Variable": ["const"] + data_selected.drop(columns=["MPCE_URP"]).columns.tolist(), 
                    "Coefficient": beta_est}))

# Summary of the model
print("\nSigma (Standard deviation of errors):", sigma_est)
print("Nu (Location parameter):", nu_est)


Coefficients:
    Variable  Coefficient
0      const          0.0
1        Age          0.0
2        Sex          0.0
3  Education          0.0
4   Religion          0.0
5      hhdsz          0.0

Sigma (Standard deviation of errors): 1.0
Nu (Location parameter): 2050.83056762287
